In [18]:
# The usual fare
# Importing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from astropy.io import fits
import matplotlib.cm as cm
import os 
import glob 
import astropy
import scipy.signal
from astropy.stats import sigma_clip
import time
from astropy import stats
from PIL import Image
import shift_methods as sm
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

In [19]:
def CustomBias(filelist,masterbias):
    '''
    CustomBias
    =======================
    Makes a custom bias frame using an image's overscan and performs a bias subtraction using it.
    
    input
    =======================
    filelist   : list of files you want to do custom bias subtract
    masterbias: the normalized master bias data
    
    returns
    =======================
    n/a
    
    files written
    =======================
    bias subtracted frame

    '''
    masterbias=masterbias
    for i in range(len(filelist)):
        header = fits.getheader(filelist[i])
        im = fits.getdata(filelist[i])
        new_name = 'b_'+filelist[i][6:-7]+'.fits'
        if im.shape == (4150,4150):
            #im_name = file[6:-7]
            overscan = np.mean(im[:,4100:4140])
            custom_bias = overscan * masterbias
            bias_subtracted = im - custom_bias
            bias_sub_overscan = bias_subtracted[:4110,:4090]
            fits.writeto(new_name,bias_sub_overscan, header,overwrite=True)
            print(new_name,'written.')

            
def norm_combine_flats(filelist):
    '''
    norm_combine_flats
    =======================
    This normalizes the flat field images
    
    inputs
    =======================
    filelist:: List of flats you need to combine in master flat
    
    returns
    =======================
    med_frame: median combined normalized frame
    
    files written
    =======================
    n/a
    '''
    n = len(filelist)
    first_frame_data = fits.getdata(filelist[0])
    imsize_y, imsize_x = first_frame_data.shape
    fits_stack = np.zeros((imsize_y, imsize_x , n), dtype = np.float32) 
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        norm_im =  im/np.median(im)# finish new line here to normalize flats
        fits_stack[:,:,ii] = norm_im
    med_frame = np.median(fits_stack, axis=2)
    return med_frame



def FileMover(filelist,foldername):
    '''
    FileMover
    =======================
    More general version of filesorter.

    inputs
    =======================
    filelist  : The list of files you want to move to a new folder
    foldername: The name of the folder you want to move the files to

    returns
    =======================
    n/a

    '''
    for file in filelist:
        if os.path.exists(file):
            pass
        else:
            print(file + "does not exist or has already been moved")
            return
        if os.path.exists(foldername):
            pass
        else:
            print("Making new directory:" + foldername)
            os.mkdir(foldername)
        destination = foldername + '/'
        print("Moving " + file + " to: ./" + destination + file)
        os.rename(file, destination + file)  
            
            
def imageshifter(imlist):
    '''
    imageshifter
    =======================
    Shifts/aligns images
    
    inputs
    =======================
    imlist: list of images you want to align to each other
    
    returns
    =======================
    n/a
    
    files written
    =======================
    shifted image
    '''
    imlist=imlist
    imlist.sort()
    # i=0
    # worklist= imlist[i:i+3]
    # for i in range(len(imlist)):
    im1,hdr1 = fits.getdata(imlist[0],header=True)
    #     # print(imlist[0])
    xshifts={}
    yshifts={}
    shift_image={}

    for index,filename in enumerate(imlist):
        imstack = np.zeros([im1.shape[0],im1.shape[1],len(imlist)])
        im = fits.getdata(filename)
        xshifts[index],yshifts[index]=sm.cross_image(im1,im,boxsize=2000)
        shift_image[index] = sm.shift_image(im,xshifts[index],yshifts[index])
        name = 's_'+filename
        fits.writeto(name,shift_image[index],hdr1,overwrite=True)
        print("Shift for image",name,"is",xshifts[index],yshifts[index])
        print('FITS file written:',name)
        print('*'*25)
    print('Shifting done!')
    
def ministack(imlist,obj,filt):
    imlist=shifted
    im1 = fits.getdata(shifted[0])
    i=0
    obj = obj
    filt = filt
    worklist=shifted[i:i+3]
    print('Starting Ministacks')
    print('*'*25)
    for i in range(len(shifted)):
        if worklist != []: 
            num = str(i)
            header = fits.getheader(worklist[0])
            medianimage = mediancombine(worklist)
            fits.writeto(num+'_'+obj+'_'+filt+'_stack.fits',medianimage,header,overwrite=True)
            print('Wrote FITS File:',num+'_'+obj+'_'+filt+'_stack.fits')
            print('*'*25)

        else:
            break

        i+=i*i+3
        worklist=imlist[i:i+3]
    print('Done Ministacking')
    
def mediancombine(filelist):
    '''
    median combine
    =======================
    Takes in a list of frames and does a medain combination on them.
    Used for biases, master flats, and basic image combination.
    
    inputs
    =======================
    filelist : The list of files you want to median combine
    
    returns
    =======================
    med_frame: The median combined frame
    '''
    n = len(filelist)
    first_frame_data = fits.getdata(filelist[0])
    imsize_y, imsize_x = first_frame_data.shape
    fits_stack = np.zeros((imsize_y, imsize_x , n), dtype = np.float32) 
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        fits_stack[:,:,ii] = im
    med_frame = np.median(fits_stack, axis=2)
    return med_frame

def filesorter(filename, foldername, fitskeyword_to_check, keyword):
    '''
    This will sort your observation data files for you. It takes in the name of a file,
    the folder you want it in, the keyword in fits to look for, and the keyword category
    in fits to look for the keyword in. Ex. sorting calibrations into new folder --
    filesorter(filename,'calibration','cal','OBJECT')
    '''
    if os.path.exists(filename):
        pass
    else:
        print(filename + " does not exist or has already been moved.")
        return
    
    header = fits.getheader(filename)
    fits_type = header[keyword]
    
    if os.path.exists(foldername):
        pass
    else:
        print("Making new directory: " + foldername)
        os.mkdir(foldername)
        
    if fits_type == fitskeyword_to_check:
        destination = foldername + '/'
        print("Moving " + filename + " to: ./" + destination + filename)
        os.rename(filename, destination + filename)  

    return

def filesorter(filename, foldername, fitskeyword_to_check, keyword):
    '''
    This will sort your observation data files for you. It takes in the name of a file,
    the folder you want it in, the keyword in fits to look for, and the keyword category
    in fits to look for the keyword in. Ex. sorting calibrations into new folder --
    filesorter(filename,'calibration','cal','OBJECT')
    '''
    if os.path.exists(filename):
        pass
    else:
        print(filename + " does not exist or has already been moved.")
        return
    
    header = fits.getheader(filename)
    fits_type = header[keyword]
    
    if os.path.exists(foldername):
        pass
    else:
        print("Making new directory: " + foldername)
        os.mkdir(foldername)
        
    if fits_type == fitskeyword_to_check:
        destination = foldername + '/'
        print("Moving " + filename + " to: ./" + destination + filename)
        os.rename(filename, destination + filename)  

    return


In [26]:
target = 'HLTau'
targdec = '18:14:25'
targra = '4:32:3.0'
filtnum = '200'
filt='Ha'
# imagetoalignto = 'E:/20190120/c8503t0205o00.fits'
masterpath = 'E:/20190122'
biaspath = 'E:/20190121/NormMasterBias.fits'
flatpath = masterpath + '/Ha/flats/HaFlat.fits'

In [27]:
masterbias = fits.getdata(biaspath)
masterflat = fits.getdata(flatpath)

In [29]:
os.chdir(masterpath+'/'+filt)
ref_position = SkyCoord(targra,targdec, unit=(u.hourangle, u.deg))
targims = []
allims=glob.glob('c*.fits')
for im in allims:
    ra = fits.getheader(im)['RASTRNG']
    dec = fits.getheader(im)['DECSTRNG']
    image_position = SkyCoord(ra,dec,unit=(u.hourangle,u.deg))
    sep = ref_position.separation(image_position)
    if sep.arcsecond < 60:
          targims.append(im)
FileMover(targims,target)

Making new directory:HLTau
Moving c8505t0110o00.fits to: ./HLTau/c8505t0110o00.fits
Moving c8505t0111o00.fits to: ./HLTau/c8505t0111o00.fits
Moving c8505t0112o00.fits to: ./HLTau/c8505t0112o00.fits
Moving c8505t0154o00.fits to: ./HLTau/c8505t0154o00.fits
Moving c8505t0155o00.fits to: ./HLTau/c8505t0155o00.fits
Moving c8505t0156o00.fits to: ./HLTau/c8505t0156o00.fits


In [33]:
os.chdir(masterpath+'/'+filt+'/'+target)
allims = glob.glob('c*.fits')
CustomBias(allims,masterbias)
b_sci = glob.glob('b_*.fits')


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'b_0110o.fits'

In [34]:
for im in b_sci:
    data = fits.getdata(im)
    header = fits.getheader(im)
    flat_fielded = data / masterflat
    name = 'f_' + im
    fits.writeto(name,flat_fielded,header,overwrite=True)
    print(name + ' has been flatfielded')
f_sci = glob.glob('f_b_*.fits')
imageshifter(f_sci)
shifted = glob.glob('s_f_b_*.fits')
ministack(shifted,target,filt)

f_b_0110o.fits has been flatfielded
f_b_0111o.fits has been flatfielded
f_b_0112o.fits has been flatfielded
f_b_0154o.fits has been flatfielded
f_b_0155o.fits has been flatfielded
f_b_0156o.fits has been flatfielded
Shift for image s_f_b_0110o.fits is 0.0 0.0
FITS file written: s_f_b_0110o.fits
*************************
Shift for image s_f_b_0111o.fits is -0.22427417480480472 -0.9108782678956686
FITS file written: s_f_b_0111o.fits
*************************
Shift for image s_f_b_0112o.fits is -0.9306226858682294 1.1310514423246332
FITS file written: s_f_b_0112o.fits
*************************
Shift for image s_f_b_0154o.fits is 3.4979294204391636 3.7134201427985545
FITS file written: s_f_b_0154o.fits
*************************
Shift for image s_f_b_0155o.fits is 0.44137873553006557 -0.2147503222549858
FITS file written: s_f_b_0155o.fits
*************************
Shift for image s_f_b_0156o.fits is 0.2258681348267828 0.04606511514941758
FITS file written: s_f_b_0156o.fits
*****************